In [1]:
import pandas as pd
from models import BalmEmbedder, Ablang2Embedder, Esm2Embedder, AntibertyEmbedder, AblangHeavyEmbedder, IgbertEmbedder
from time import time
import os

In [2]:
sabdab_df = pd.read_parquet('ablangpdb_renameddatasets.parquet').drop(columns="EMBEDDING")
dms_df = pd.read_parquet('ablangrbd_renameddatasets.parquet').drop(columns="EMBEDDING")
print(dms_df.shape, sabdab_df.shape)
print(dms_df.columns.to_list(), sabdab_df.columns.to_list())

(3041, 24) (1909, 42)
['Unnamed: 0', 'NAME', 'EPITOPE', 'HV', 'HJ', 'LV', 'LJ', 'HC_AA', 'LC_AA', 'DONOR', 'CDRH3', 'CDRL3', 'EPITOPE_REGION', 'AG_BROAD', 'AG_NARROW', 'EMBEDDING_IDX', 'CLONOTYPE', 'DATASET', 'CLASS', 'INDEX', 'PREPARED_HC_SEQ', 'PREPARED_LC_SEQ', 'EPITOPE_LABELS', 'SYNTHETIC'] ['Column1', 'AG_CLUSTER', 'AG_AA', 'CLAN', 'PFAM_PLUS', 'NAME_x', 'HV', 'HJ', 'CDRH3', 'HC_AA', 'LV', 'LJ', 'CDRL3', 'LC_AA', 'CLONE', 'NUM_TRAIN_PAIRS', 'NUM_TRAIN_SAME_PFAM_DIF_EP', 'NUM_TRAIN_OVERLAPPING_EP', 'DATASET', 'PREPARED_HC_SEQ', 'PREPARED_LC_SEQ', 'H_TOKENS', 'L_TOKENS', 'H_ATTENTION_MASK', 'L_ATTENTION_MASK', 'SAME_AG_POSSIBLE', 'SAME_EP_POSSIBLE', 'NN_ACTUAL', 'NN_ACTUAL_PFAM', 'NN_ACTUAL_COS_SIM', 'NN_ACTUAL_PREDICTED_COS_SIM', 'NN_ACTUAL_PREDICTED_RANK', 'NN_GUESS', 'NN_GUESS_PFAM', 'NN_GUESS_COS_SIM', 'NN_GUESS_ACTUAL_LABEL', 'NN_GUESS_RANK', 'CORRECT_NN_EP_GUESS', 'CORRECT_NN_PFAM_GUESS', 'PLOTTING_AG', 't-SNE_1', 't-SNE_2']


In [3]:
balm_model_dir = "../../../pretrained_models/BALM-paired_LC-coherence_90-5-5-split_122222"
device = "cuda"


            if model_name == "BALM":
1&2: Fail. Path error
            elif model_name == "AbLang2":
1&2: Fail.    model.eval() call
            elif model_name == "ESM-2":
1 Success
2. Fail. Cuda
            elif model_name == "AntiBERTy":
1 Success
2. Fail. Cuda
            elif model_name == "AbLang-Heavy":
1 Fail. Cuda
2. Fail. Cuda
            else:  # model_name == "IgBERT":
1. Fail. Cuda
2 Fail. Cuda

In [4]:
# 2. Generate Embeddings
skip_existing_embeds = True
datasets = [("SABDAB", sabdab_df), ("DMS", dms_df)]
models = ["BALM", "AbLang2", "ESM-2", "AntiBERTy", "AbLang-Heavy", "IgBERT"]
# datasets = [("SABDAB", sabdab_df)]
# models = ["AbLang2", "AbLang-Heavy"] #, "IgBERT"]
for dataset, starting_df in datasets:  # for 2 datasets
    for model_name in models:  # Use 6 models for embedding
        output_file = f"{dataset.lower()}_embeddedby_{model_name.lower()}.parquet"
        if os.path.exists(output_file) and skip_existing_embeds:  # Only do it if file doesn't exist
            print("Skipping", output_file)
            continue
        
        t1 = time()
        try:
            print("Starting with model ", model_name)
            
            df = starting_df.copy() # Make sure not to overwrite original

            if model_name == "BALM":
                embedder = BalmEmbedder(model_directory=balm_model_dir, device=device)
            elif model_name == "AbLang2":
                embedder = Ablang2Embedder(device=device)
            elif model_name == "ESM-2":
                embedder = Esm2Embedder(device=device)
            elif model_name == "AntiBERTy":
                embedder = AntibertyEmbedder()
            elif model_name == "AbLang-Heavy":
                embedder = AblangHeavyEmbedder(device=device)
            else:  # model_name == "IgBERT":
                embedder = IgbertEmbedder(device=device)

            print("Embedder set up, going on to embed now")
            embeddings = embedder.embed(df)
            df["EMBEDDING"] = list(embeddings)
            del embeddings, embedder  # Clean up memory

            # 3. Save Data
            print(f"Saving embeddings to {output_file}...")
            df.to_parquet(output_file)
            print("Suceeded with model", model_name, f"Shape {df.shape}\n\n")

        except Exception as e:
            print(f"Model {model_name} failed with exception: {e}")

Skipping sabdab_embeddedby_balm.parquet
Skipping sabdab_embeddedby_ablang2.parquet
Skipping sabdab_embeddedby_esm-2.parquet
Skipping sabdab_embeddedby_antiberty.parquet
Skipping sabdab_embeddedby_ablang-heavy.parquet
Skipping sabdab_embeddedby_igbert.parquet
Skipping dms_embeddedby_balm.parquet
Skipping dms_embeddedby_ablang2.parquet
Skipping dms_embeddedby_esm-2.parquet
Starting with model  AntiBERTy
Embedder set up, going on to embed now


100%|██████████| 12/12 [00:14<00:00,  1.19s/it]


Saving embeddings to dms_embeddedby_antiberty.parquet...
Suceeded with model AntiBERTy Shape (3041, 25)


Skipping dms_embeddedby_ablang-heavy.parquet
Skipping dms_embeddedby_igbert.parquet


In [7]:
from glob import glob
for x in ["sabdab", "dms"]:
    for fname in glob(f"{x}*.parquet"):
        df = pd.read_parquet(fname)
        print(fname, df.shape)

sabdab_embeddedby_ablang-heavy.parquet (1909, 43)
sabdab_embeddedby_ablang2.parquet (1909, 43)
sabdab_embeddedby_ablangpre.parquet (1909, 43)
sabdab_embeddedby_ablangrbd.parquet (1909, 43)
sabdab_embeddedby_antiberty.parquet (1909, 43)
sabdab_embeddedby_balm.parquet (1909, 43)
sabdab_embeddedby_esm-2.parquet (1909, 43)
sabdab_embeddedby_igbert.parquet (1909, 43)
sabdab_embeddedby_parapred.parquet (1909, 48)
dms_embeddedby_ablang-heavy.parquet (3041, 25)
dms_embeddedby_ablang2.parquet (3041, 25)
dms_embeddedby_ablangpdb.parquet (3041, 25)
dms_embeddedby_ablangpre.parquet (3041, 25)
dms_embeddedby_antiberty.parquet (3041, 25)
dms_embeddedby_balm.parquet (3041, 25)
dms_embeddedby_esm-2.parquet (3041, 25)
dms_embeddedby_igbert.parquet (3041, 25)
dms_embeddedby_parapred.parquet (3041, 30)


In [2]:
from tqdm import tqdm
batch_size = 2
sequences = ["a", "b", "c", "d", "E"]
for i in tqdm(range(0, len(sequences),batch_size)):
        batch_sequences = sequences[i:i+batch_size]
        print(batch_sequences)

100%|██████████| 3/3 [00:00<00:00, 20197.29it/s]

['a', 'b']
['c', 'd']
['E']


In [ ]:
# ["BALM", "AbLang2", "ESM-2", "AntiBERTy", "AbLang-Heavy", "IgBERT"]
failed = ["BALM", "AbLang2"]
succeeded = ["?ESM-2"]